# Results exporter
Notebook to easily load a sacred experiment and create a CSV file ready to be used in Latex.

In [26]:
import sys
sys.path = ['..'] + sys.path
import pandas as pd
import numpy as np
import glob, sys, json, re
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sts
from sacred_notebook_utils import plot_mean_ci, plot_all
from baselines.common.sacred_utils import load_runs, filter_runs
import scipy.stats as sts

In [2]:
runs = load_runs('../../../sacred_data/sacred_runs_master/')
print("Loaded runs:", len(runs.keys()))
final_runs = load_runs('../../../sacred_data/sacred_runs_final/')
print("Loaded final runs:", len(final_runs.keys()))

Loaded runs: 572
Loaded final runs: 704


In [3]:
pdisnn_runs_local = load_runs('../sacred_pdisnn/')
print("Loaded pdisnn runs:", len(pdisnn_runs_local.keys()))
# Merge the runs
total_runs = {}
for key, value in runs.items():
    total_runs[str(key) + '-base'] = value
for key, value in final_runs.items():
    total_runs[str(key) + '-final'] = value
for key, value in pdisnn_runs_local.items():
    total_runs[str(key) + '-local'] = value

Loaded pdisnn runs: 213


## Load the experiment we need

In [67]:
query = {
    'run.status': 'COMPLETED',
    'config.center': False,
    'config.gamma': 1.0,
    'config.clipping': False,
    'config.natural': False,
    'config.policy': 'linear',
    'config.iw_norm': 'none',
    'config.entropy': 'none',
    'config.env': 'rllab.inverted-pendulum',
    'config.iw_method': 'pdis',
    'config.delta': 0.9999,
    'config.bound': 'pdis-max-d2',
    'config.num_episodes': 100
}
filtered_runs = filter_runs(query, total_runs)
print("Experiments:", len(filtered_runs.keys()))
print("Seeds:", ", ".join(map(str, [filtered_runs[key]['config']['seed'] for key in filtered_runs.keys()])))

Experiments: 25
Seeds: 481, 236, 247, 245, 904, 109, 570, 178, 307, 746, 42, 963, 689, 712, 100, 153, 662, 947, 15, 10, 8, 315, 545, 160, 950


## Extract the data we need for the CSV

In [68]:
runs = list(filtered_runs.keys())
df = []
for run_key in runs:
    episodes = filtered_runs[run_key]['metrics']['EpisodesSoFar']['values']
    rewards = filtered_runs[run_key]['metrics']['EpRewMean']['values']
    df.append(pd.Series(rewards, episodes))
performance = pd.concat(df, axis=1)
values = performance.values
initial_columns = list(performance)
performance['mean'] = np.mean(values, axis=1)
performance['std'] = np.std(values, axis=1)
low, up = sts.t.interval(0.95, values.shape[1]-1, loc=np.mean(values, axis=1), scale=sts.sem(values, axis=1))
performance['low'] = low
performance['up'] = up
performance.drop(columns=initial_columns, inplace=True)
performance.reset_index(inplace=True)
performance.rename(columns={'index': 'iteration'}, inplace=True)
performance['iteration'] -= 100
performance = performance.iloc[::5]
print(performance.shape)
performance.head()

(100, 5)


,iteration,mean,std,low,up
0,0,-151.972289,2.707095,-153.112765,-150.831813
5,500,-128.641413,7.564690,-131.828353,-125.454473
10,1000,-122.992345,6.363244,-125.673126,-120.311564
15,1500,-117.489331,7.443761,-120.625324,-114.353337
20,2000,-115.592714,7.828478,-118.890785,-112.294642


## Create the CSV

In [69]:
performance.to_csv('inverted-pendulum-pdis-pois.csv', index=False)